# 背景
针对部分订单广告主会有TA要求（客户往往以三方数据考核TA），为了使订单完成目标KPI，投放人群需与第三方的人群标签一致。

人群包分为两种：
1. TA包
2. segment包

TA包用于实时投放中的替换，将设备A替换为TA包中满足人群标签的设备B。

segment包用于实时投放中的过滤，即挑选出流量中存在于segment包中的设备，在FancyDSP中对选中流量进行参竞，在FTX中对选中流量再发送给其他DSP

# TA包

## 逻辑

## 六个字段
设备id，

设备类型： 1 imei 2 idfa 3 oaid 4 ott，

mac '528C8E6CD4A3C6598999A0E9DF15AD32' 写死，

geoid，

广告主id   不限广告主则填零，

uaid  需排除未解析出uaid的设备
## 推TA包脚本使用方法
step 1: 登陆da账号  
step 2: `cdb` 进入bin目录  
step 3: `sh push_ta_package.sh -h` 查看脚本使用方法
## 检查TA包是否有量
step 1: 登陆da账号  
step 2: `cdta` 进入ta包目录  
step 3: `wc -l ta包__时间戳`  
## 正常推TA包
step 1: 登陆da账号  
step 2: `cdb` 进入bin目录  
step 3: `sh push_ta_package.sh -t ta包id -l 20000000`   (默认推20000000条)  
step 4: 登陆fancy media 将对应ta包状态改为可用  
## 推TA包超内存
step 1: 登陆da账号  
step 2: `cdb` 进入bin目录  
step 3: `sh push_ta_package.sh -t ta包id-l 20000000`   (默认推20000000条)  
step 4: 发现超内存  
step 4: 登陆EasyWeb 找到进程（with开头）  
step 5: 复制EasyWeb 中的sql在内层sql中limit 200000000（2亿),防止底层数据过多超过sql内存
step 6: `cdta` 进入ta包目录  
step 7: `presto --execute " sql语句 " --output-format TSV > ta包__时间戳;` （e.g. 1850__202103191203)  
step 8: `cdb` 进入bin 目录  
step 9: `sh update_ta_snapshot.sh` 更新TA包  
step 10: 登陆fancy media 将对应ta包状态改为可用
## 推有条件TA包（投放过设备或参竞过设备需去离线表中将设备信息提取出来）
step 1: 登陆da账号  
step 2: `cdta` 进入ta包目录  
step 3: 按照条件写好sql  （例子为在订单2464曝光过却为在订单2805曝光过的设备）
> `select distinct a.ext_user_id,
>            case
>                when a.ext_device_type = 4 then 4
>                else pf end         jixing,
>            case
>                when a.ext_device_type = 4 then a.ext_mobile_mac
>                else '528C8E6CD4A3C6598999A0E9DF15AD32'
>                end                 mac,
>            case
>                when d.geo_id is null then cast(a.geoid as varchar)
>                else d.geo_b_id end geoid,
>            a.ext_advertiser_id,
>            a.ua_id_v2
> from (
>     select ext_user_id
>          , case
>                when ext_platform = 'Android' and length(ext_user_id) = 32 then 1
>                when ext_platform = 'IOS' then 2
>                else 3 end  pf
>          , ext_device_type
>          , ext_advertiser_id
>          , ua_id_v2
>          , ext_mobile_mac
>          , max(ext_geo_id) geoid
>     from dwd.d_ad_impression
>     where ext_order_id = 2464
>       and ext_advertiser_id = 2003513
>       and thisdate >= '2020-08-06'
>       and thisdate <= '2020-10-15'
>     group by 1, 2, 3, 4, 5, 6
> ) a
>     left join
> (
>     select distinct ext_user_id
>     from dwd.d_ad_impression
>     where ext_order_id = 2805
>       and ext_advertiser_id = 2003584
>       and thisdate >= '2020-11-13'
>       and thisdate <= '2020-12-31'
> ) b
> on a.ext_user_id = b.ext_user_id
>     join
> u6rds.ad_fancy.base_ua_id_info c
> on cast(a.ua_id_v2 as varchar) = c.ua_id
>     left join
> (
>     select geo_id, geo_b_id
>     from dim.dim_area
> ) d
> on a.geoid = d.geo_id
> where b.ext_user_id is null`

step 4: `presto --execute " sql语句 " --output-format TSV > ta包__时间戳;` （e.g. 1850__202103191203)  
step 5: `cdb` 进入bin 目录  
step 6: `sh update_ta_snapshot.sh` 更新TA包  
step 7: 登陆fancy media 将对应ta包状态改为可用  
## 推阿里条件TA包 
阿里的设备id优先级与我方不同，他们要求安卓10以下用imei，安卓10及以上用oaid，苹果用idfa

step 1: 登陆da账号    
step 2: `cdb` 进入bin目录  
step 3: `sh push_ta_package.sh -t ta包id -l 20000000`   (默认推20000000条)  
step 4: `cdta` 进入ta包目录  
step 5: `ll` 查看ta包具体名称  
step 6: `awk '{if(substr($6,9,4)<2112 && $2==1) {print $0}}' ta包名称 > linshi1`  筛选出满足安卓10且设备类型为imei的设备
step 7: `awk '{if(substr($6,9,4)>=2112 && substr($6,9,4)<4000 && $2==3) {print $0}}' ta包名称 > linshi2`  筛选出满足安卓10及以上且设备类型为oaid的设备
step 8: `awk '{if(substr($6,9,4)>4000 && $2==2) {print $0}}' ta包名称 > linshi3`  筛选出满足苹果且设备类型为idfa的设备
step 9: `wc -l linshi*` 查看是否每个条件都有量  
step 10: `cat linshi1 linshi2 linshi3 > ta包__新的时间戳`  将上述三个临时包汇总成一个ta抱
step 11: `rm linshi* ta包名称`  删除脚本跑出的ta包
step 12: `cdb` 进入bin 目录  
step 13: `sh update_ta_snapshot.sh` 更新TA包  
step 14: 登陆fancy media 将对应ta包状态改为可用
## OTT条件TA包
step 1: 登陆da账号   
step 2: 找到要求的地域geo_code  
step 3: `cdb` 进入bin目录    
step 4: `sh push_ta_package.sh -t ta包id -l 20000000`   
step 5: 登陆EasyWeb 找到进程（with开头）   
step 6: 复制sql 中的 geo_code   （上述过程用于提取ta包所投地域信息）
step 7: `cdta` 进入ta包目录  
step 8: 跑下面这个sql 替换geo_code   
`presto --execute "
-- 0 代表所有广告主
select user_id, 4, mac, geo_code, 0, ua_id
from rpt.base_device_ua_id_v2
--thisdate写前天日期（一般情况）
where thisdate = '2021-03-07'
  and ua_id not in ('0', '-1')
  and uid_type = '4' -- OTT
  --去dim_area里查地名(geo_b_code)
  and geo_code in ('156024000011','156024000013')
group by 1, 2, 3, 4, 5, 6 limit 20000000
" --output-format TSV > ta包__时间戳;`  
step 9: `cdb` 进入bin 目录  
step 10: `sh update_ta_snapshot.sh` 更新TA包  
step 11: 登陆fancy media 将对应ta包状态改为可用







# Segment 包
## 新建segment包
step 1: `https://adm.fancydsp.com/segment`   
step 2: 填写包名称，其他选项参考之前建好的包  
## 推TA包脚本使用方法
step 1: 登陆da账号  
step 2: `cdb` 进入bin目录  
step 3: `sh push_segment.sh -h` 查看脚本使用方法
## 推包
step 1: 登陆da账号  
step 2: cdb 进入bin目录  
step 3: sh push_segment.sh -s segment包 （根据要求使用，推文件或推sql）

step 4: 去网页更改uv数    

## TA包 正常推范例
### [1875]好域-测试-3y23  为屹，帮忙跑下这个包~
![zc1](../imgs/推包/zc1.png)
![zc2](../imgs/推包/zc2.png)
![zc3](../imgs/推包/zc3.png)
![zc4](../imgs/推包/zc4.png)
## TA包 阿里条件范例
### Dear 为屹：辛苦按阿里推包逻辑推以下TA包：[1874]FTX-阿里_新势力周-大白_韩剧TV开屏-3.24-3.26-----投放媒体点位：韩剧TV_开屏
![al1](../imgs/推包/al1.png)
![al2](../imgs/推包/al2.png)
![al3](../imgs/推包/al3.png)
![al4](../imgs/推包/al4.png)
![al5](../imgs/推包/al5.png)
![al6](../imgs/推包/al6.png)
![al7](../imgs/推包/al7.png)